## Machine Learning Exercise 3: Bias and Variance ##

**Bias** refers to the error introduced by approximating a complex real-world problem with a simplified model, while **variance** refers to the model's sensitivity to fluctuations in the training data. A linear regression model has high bias and low variance; it makes strong assumptions about the data (linearity) but is stable across different datasets. If these strong assumptions are not correct, there will be places where it systematically overestimates or underestimates. In contrast, a decision tree model has low bias and high variance; it can capture complex patterns but is prone to overfitting, especially if deep and unpruned. This means that it can start to memorize the training data rather than capturing patterns that generalize.

In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import(
    LogisticRegression, 
    LinearRegression,
    RidgeCV,
    LassoCV
)
from sklearn.metrics import( 
    confusion_matrix, 
    RocCurveDisplay, 
    classification_report, 
    r2_score, 
    mean_squared_error, 
    root_mean_squared_error, 
    mean_absolute_error, 
    mean_absolute_percentage_error,
    make_scorer
)
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [4]:
data = pd.read_csv('../data/kc_house_data.csv')

data.sample(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
3308,1938400300,20140708T000000,245000.0,4,2.25,2600,6390,1.0,0,0,...,8,1390,1210,1978,0,98023,47.3174,-122.366,2110,6700
15521,3052700225,20140814T000000,727160.0,7,3.75,2310,5000,2.0,0,0,...,8,2310,0,1984,0,98117,47.6781,-122.376,1360,1552
13162,1072100085,20140514T000000,310000.0,3,1.00,1480,7830,1.0,0,0,...,7,1480,0,1952,0,98133,47.7703,-122.336,1450,7830


1. Fit a linear regression model to the housing data, using sqft_living to predict price. Check the mean squared error on the training data and the test data.

In [6]:
model = LinearRegression()

X = data[['sqft_living']]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n' 
      f'MSE_train: {mean_squared_error(X_train, y_train)}')

MSE_test: 66079560485.167015
MSE_train: 427016656868.78235

2. Repeat this but with a [DecisionTreeRegresor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). Again check the mean squared error on the training data and the test data. How does what you see differ from the linear regression model?

In [8]:
model = DecisionTreeRegressor(criterion = 'squared_error')

X = data[['sqft_living']]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n' 
      f'MSE_train: {mean_squared_error(X_train, y_train)}')

MSE_test: 73538459276.97156
MSE_train: 427016656868.78235

One way of avoiding overfitting is by restricting the flexibility of the model. We can do this with a decision tree by restricting the number of splits that it can perform. 

3. Fit a DecisionTreeRegressor where you restrict the max_depth to 5. Again check the mean squared error on the training data and the test data. What do you notice now?

In [10]:
model = DecisionTreeRegressor(criterion = 'squared_error', max_depth = 5)

X = data[['sqft_living']]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n' 
      f'MSE_train: {mean_squared_error(X_pred_train, y_train)}')

MSE_test: 62926561451.64904
MSE_train: 427016656868.78235

When working with machine learning models, we often have to balance bias and variance. This is called the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff). One method of this is through [regularization](https://www.ibm.com/think/topics/regularization), where we restrict the complexity of the model, adding some bias but reducing the variance, which can lead to a lower mean squared error on the test set.

Lasso and ridge regression do this by adding a penalty term based on the size of the coefficients. Smaller coefficients mens that the model has less flexibility. The neat thing about these types of models is that they determine how to allocate the coefficients automatically as part of the model fitting process, so we can start with a large set of potential predictors and allow the model fitting to determine which ones to focus on.

For the next part of the exercise, we'll see how we can add complexity to our model but control the complexity through regularization.

4. So far, we've only been predicting based off of the square footage of living space. Fit a new linear regression model using all variables besides id, date, price, and zipcode. How well does this model perform on the test set compared to the model with just square footage of living space?

In [32]:
model = LinearRegression()

data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n'
      f'MSE_train:{mean_squared_error(y_train, y_pred_train)}')

MSE_test: 41757655724.75456
MSE_train: 40804822162.2284

5. Try fitting a lasso and ridge model. Becuase lasso and ridge have penalty terms based on the size of the coefficients, and the size of thae coefficients depends on the scale of the variable, you'll want to scale the features first so that they are on comparable scales. Create a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object where the first step is applying a [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and the second step is either a lasso or ridge model. Because these models have a hyperparameter controlling regularization strength, you'll want to use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) models, which will select values for the regularization strength using cross-validation.

In [60]:
# Lasso Regression
data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

pipe = Pipeline(steps = [('scaler', StandardScaler()),
         ('lasso', LassoCV())])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

y_pred_train = pipe.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n'
      f'MSE_train:{mean_squared_error(y_train, y_pred_train)}')

MSE_test: 41770331754.19049
MSE_train: 40806129474.58575

In [62]:
# Ridge Regression
data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

pipe = Pipeline(steps = [('scaler', StandardScaler()),
         ('ridge', RidgeCV())])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

y_pred_train = pipe.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n'
      f'MSE_train:{mean_squared_error(y_train, y_pred_train)}')


MSE_test: 41757913805.375755
MSE_train: 40804846790.29052

You likely didn't see much difference between the regular linear regression model and the lasso or ridge model. Let's see what happens if we add more complexity through feature interactions. We can capture more complex relationships between the predictor variables and the target variable by multiplying the predictors together before fitting the model. For example, the interaction between sqft_living and bedrooms will let the model capture if the impact of square footage depends on the number of bedrooms.

6. Add [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to your pipeline after the standard scaler. Try using degree 2 features. How does this change the performance of the regular linear regression model, the lasso model, and the ridge model? 

In [82]:
# Lasso Regression polynomial
data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

lasso_pipe = Pipeline(steps = [('scaler', StandardScaler()),
                         ('poly', PolynomialFeatures(degree=2)),
                         ('lasso', LassoCV())])

lasso_pipe.fit(X_train, y_train)
y_pred = lasso_pipe.predict(X_test)

y_pred_train = lasso_pipe.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n'
      f'MSE_train:{mean_squared_error(y_train, y_pred_train)}')

MSE_test: 25216536774.624573
MSE_train: 26353345246.66341

In [84]:
#Ridge regression polynomial
data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

ridge_pipe = Pipeline(steps = [('scaler', StandardScaler()),
                         ('poly', PolynomialFeatures(degree=2)),
                         ('ridge', RidgeCV())])

ridge_pipe.fit(X_train, y_train)
y_pred = ridge_pipe.predict(X_test)

y_pred_train = ridge_pipe.predict(X_train)

print(f'MSE_test: {mean_squared_error(y_test, y_pred)}', end = '\n'
      f'MSE_train:{mean_squared_error(y_train, y_pred_train)}')

MSE_test: 25216536774.624573
MSE_train: 25611197675.461464

The lasso penalty tends to cause some coeffients to zero out, so it can be viewed as a method of automatic feature selection.

7. Look at the set of coefficients for the lasso model. What percentage of the coefficients are zero? What are the largest non-zero coefficients?

In [122]:
lasso_cv = lasso_pipe[-1]

arr = lasso_cv.coef_

zeroes = np.count_nonzero(arr==0)
values = len(arr)

percent = str(round(zeroes / values * 100)) + '%'

percent

'49%'

In [144]:
import heapq

pd.DataFrame(heapq.nlargest(10, arr))

,0
0,96953.631743
1,95077.950449
2,70301.434205
3,39077.771105
4,38574.730072
5,25041.628455
6,22854.447805
7,18861.158245
8,17979.925325
9,17303.867863


In [ ]:
coefficients = pd.DataFrame({
'variable': pipe[:-1].get_feature_names_out(),
'coefficient': pipe['lr'].coef_
})

8. A new hyperparameter that we have is the degree of the polynomial we're using. So that we're not overfitting to the test set, we need to use cross-validation to select this value. Set up a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to try out polynomial degrees from 1 to 3 and to try LinearRegression, LassoCV, and RidgeCV models. Use 'neg_mean_squared_error' as the error_score. Which combination does it find does the best?

**If you've reached this point, let your instructors know so that they can check in with you.**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [217]:
#linear regression grid search

data_new = data.drop(columns = ['id', 'date', 'price', 'zipcode'])

X = data_new
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=321)

models = [
    LinearRegression(),
    RidgeCV(),
    LassoCV(cv=5, max_iter = 10000)
]

param_grid = {
    'model': models,
    'polynomialfeatures__degree': [1, 2, 3]
}

best_neg_mse = float('inf')
best_params = None

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('polynomialfeatures', PolynomialFeatures()),
    ('model', model)
])
    
grid_search = GridSearchCV(pipeline, param_grid, scoring = 'neg_mean_squared_error', cv = 5)
grid_search.fit(X_train, y_train)

if grid_search.best_score_ < best_neg_mse:
    best_neg_mse = grid_search.best_score_
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

y_pred = best_estimator.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Best Negative Mean Squared Error: {best_neg_mse}")
print(f"Test Mean Squared Error: {test_mse}")

Best Parameters: {'model': LassoCV(cv=5, max_iter=10000), 'polynomialfeatures__degree': 2}
Best Negative Mean Squared Error: -29157768309.902996
Test Mean Squared Error: 25135804989.076473


## Stretch Goals: ##

1. With home prices, there are some extremely large values for price, which can overly-influence the mean squared error calculation. See what happens if you optimize for mean absolute error instead. Alternatively, try using a [TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) to predict the log of price instead of the raw price.

**Bonus Exercise** We've seen how a decision tree model has more flexibility, which mean higher variance compared to a linear regression model. One way of understanding variance is that variance describes how sensitive the model is to the training data. A model with high variance can produce vastly different predictions when trained on different subsets.

In this bonus exercise, you'll get to see this in action.

Generate 1000 different linear regression fits, where you only use sqft_living as the predictor variable. For each fit, choose a random sample from the full dataset of size 1000. 
Using the np.linspace function, generate a grid of 100 equally-spaced points between 500 and 3000 and generate predictions on those points. Plot the mean prediction, the 5th percentile, and the 95th percentile for the predictions from these thousand models. Repeat this for a decision tree model. Then do it for a decision tree model with a max_depth of 5.

How do these predictions differ? Which have the most variability?